In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import os
import numpy as np
import torch
print(torch.__version__)

1.9.0+cu111


In [4]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# The GPU id to use, "0" to  "7"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
!git clone https://github.com/facebookresearch/detr.git

fatal: destination path 'detr' already exists and is not an empty directory.


In [6]:
import sys
sys.path.append(os.path.join(os.getcwd(), "detr/"))
# print(sys.path)

In [7]:
def get_device() -> tuple:
    """Get Device type

    Returns:
        tuple: Device type
    """
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    return (use_cuda, device)

use_cuda, device = get_device()

In [8]:
import random
import shutil
import sys

import cv2

from categories_meta import COCO_CATEGORIES, COCO_NAMES
from panopticapi.utils import id2rgb, rgb2id
import panopticapi
from PIL import Image, ImageDraw, ImageFont
import requests
import json
import io
import math
import matplotlib.pyplot as plt
# %config InlineBackend.figure_format = 'retina'
import itertools

import torch
from torch import nn
import torchvision.transforms as T
import numpy as np

torch.set_grad_enabled(False)
# Create Original Segmented Image
import overlay_custom_mask
import convert_to_coco

from categories_meta import COCO_CATEGORIES, NEW_CATEGORIES, MAPPINGS, INFO, LICENSES, cat2id, id2cat
import coco_creator_tools

import datetime
import time
import json
import traceback

In [9]:
# standard PyTorch mean-std input image normalization
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load detr model
model, postprocessor = torch.hub.load('detr', 'detr_resnet101_panoptic', source='local', pretrained=True, return_postprocessor=True, num_classes=250)
# Convert to eval mode
model = model.to(device)
model.eval()

print("Model Loaded")

Model Loaded


In [28]:
# import glob

# category_paths = glob.glob('/home/ammar/data/construction/*')

category_paths = [
    '/home/ammar/data/construction/hydra_crane',
    '/home/ammar/data/construction/split_units',
    '/home/ammar/data/construction/wheel_loader',
    '/home/ammar/data/construction/dump_truck_tipper_truck',
    '/home/ammar/data/construction/transit_mixer',
    '/home/ammar/data/construction/glass_wool',
    '/home/ammar/data/construction/rmu_units',
    '/home/ammar/data/construction/chiller',
    '/home/ammar/data/construction/adhesives',
    '/home/ammar/data/construction/structural_steel_channel',
    '/home/ammar/data/construction/sanitary_fixtures',
    '/home/ammar/data/construction/aac_blocks',
    '/home/ammar/data/construction/control_panel',
    '/home/ammar/data/construction/vrf_units',
    '/home/ammar/data/construction/wood_primer',
    '/home/ammar/data/construction/rcc_hume_pipes',
    '/home/ammar/data/construction/hoist',
    '/home/ammar/data/construction/emulsion_paint',
    '/home/ammar/data/construction/threaded_rod',
    '/home/ammar/data/construction/switch_boards_and_switches',
    '/home/ammar/data/construction/smoke_detectors',
    '/home/ammar/data/construction/vcb_panel',
    '/home/ammar/data/construction/rmc_batching_plant',
    '/home/ammar/data/construction/fine_aggregate',
    '/home/ammar/data/construction/hollow_concrete_blocks',
    '/home/ammar/data/construction/junction_box',
    '/home/ammar/data/construction/marble',
    '/home/ammar/data/construction/pipe_fittings',
    '/home/ammar/data/construction/lime',
    '/home/ammar/data/construction/vitrified_tiles',
    '/home/ammar/data/construction/water_tank',
    '/home/ammar/data/construction/concrete_mixer_machine',
    '/home/ammar/data/construction/metal_primer',
    '/home/ammar/data/construction/river_sand',
    '/home/ammar/data/construction/distribution_transformer',
    '/home/ammar/data/construction/fire_buckets',
    '/home/ammar/data/construction/concrete_pump',
    '/home/ammar/data/construction/refrigerant_gas',
    '/home/ammar/data/construction/grader',
    '/home/ammar/data/construction/ahus',
    '/home/ammar/data/construction/interlocked_switched_socket',
    '/home/ammar/data/construction/skid_steer_loader',
    '/home/ammar/data/construction/enamel_paint',
    '/home/ammar/data/construction/texture_paint',
    '/home/ammar/data/construction/fire_extinguishers',
    '/home/ammar/data/construction/aluminium_frames_for_false_ceiling',
    '/home/ammar/data/construction/hot_mix_plant',
    '/home/ammar/data/construction/cu_piping'
]

In [29]:
def convert(o):
    if isinstance(o, np.generic): return o.item()  
    raise TypeError

In [30]:
ROOT_DIR = './data'

processing_file = ""
processing_data = []

image_id = 1
annotation_id = 1
segment_id = 1

GLOBAL_COCO = {
    "licenses": LICENSES,
    "info": INFO,
    "categories": NEW_CATEGORIES,
    "annotations": [],
    "images": []
}

GLOBAL_PANOPTIC = {
    "licenses": LICENSES,
    "info": INFO,
    "categories": NEW_CATEGORIES,
    "annotations": [],
    "images": []
}

############################ Create DATASET ################################

# run through all folders in dataset
for category_path in category_paths:
    # store starting time
    start = time.time()
    # get category name
    category_name = category_path.split("/")[5]
    print("Processing Category:", category_name)
    # open category coco file
    with open(os.path.join(category_path, "coco.json"), "r") as coco_file:
        category_coco = json.load(coco_file)
        
    images_root = os.path.join(category_path, 'images')
        
    # Process all images
    ## 1. Create a temp json which contains each image and its annotations
    ## 2. Run over this list
    ### 1. Copy this image as .jpg in GLOBAL_DIR
    ### 2. Find all segments for this image
    ### 3. Create new anotation segment which includes annotations from custom classes
    
    TEMP_COCO_IMAGES = {}
    
    # Run over all images
    for im in category_coco["images"]:
        im['annotations'] = []
        TEMP_COCO_IMAGES[im['id']] = im
        
    for ann in category_coco["annotations"]:
        TEMP_COCO_IMAGES[ann['image_id']]["annotations"].append(ann)
        
    for i, image_coco in TEMP_COCO_IMAGES.items():
        # get image path
        ## This data can be used further for logging if failed while processing
        processing_file = os.path.join(images_root, image_coco['file_name'])
        processing_data = image_coco
        output_file_name = category_name + "_" + str(image_id) + ".jpg"
        output_file_path = os.path.join(ROOT_DIR, "images", output_file_name)
        
        output_mask_name = category_name + "_" + str(image_id) + ".png"
        output_mask_path = os.path.join(ROOT_DIR, "masks", output_mask_name)
        
        try:

            # Read this image and get shape of image
            imo = Image.open(processing_file).convert('RGB')

            try:
                h, w, c = np.array(imo).shape
            except:
                h, w = np.array(imo).shape
                c = 1

            # if no of channels != 3, open the image and convert it to 3 channel - RGB
            if c == 4 or c == 1:
                imo = imo.convert('RGB')
                h, w, c = np.array(imo).shape

            # Create a copy of image this will be used for further processing
            im = imo.copy()

            # Apply transform and convert image to batch
            # mean-std normalize the input image (batch-size: 1)
            img = transform(im).unsqueeze(0).to(device)  # [h, w, c] -> [1, c, ht, wt]

            # Generate output for image
            out = model(img)

            # Generate score
            # compute the scores, excluding the "no-object" class (the last one)
            scores = out["pred_logits"].softmax(-1)[..., :-1].max(-1)[0]

            # threshold the confidence
            keep = scores > 0.85

            # Keep only ones above threshold
            pred_logits, pred_boxes = out["pred_logits"][keep][:, :len(
                COCO_NAMES) - 1], out["pred_boxes"][keep]

            # the post-processor expects as input the target size of the predictions (which we set here to the image size)
            result = postprocessor(out, torch.as_tensor(img.shape[-2:]).unsqueeze(0))[0]

            # The segmentation is stored in a special-format png
            panoptic_seg = Image.open(io.BytesIO(result['png_string'])).resize((w, h), Image.NEAREST)
            # (wp, hp) = panoptic_seg.size
            panoptic_seg = np.array(panoptic_seg, dtype=np.uint8).copy()

            # We retrieve the ids corresponding to each mask
            panoptic_seg_id = rgb2id(panoptic_seg)
            
            ## Merge predicted annotations
            
            # 1. Get Mapping from predicted id to new ids
            ## 1. Merge segment ids
            unique_category_id = []
            for i, segment in enumerate(result['segments_info']):
                result['segments_info'][i]["category_id"] = MAPPINGS[result['segments_info'][i]["category_id"]]
                if result['segments_info'][i]["category_id"] not in unique_category_id:
                    unique_category_id.append(result['segments_info'][i]["category_id"])
            
            # Sort array
            unique_category_id.sort()
            
            unique_category_id_to_id =  {category_id: i for i, category_id in enumerate(unique_category_id)}
            unique_id_to_category_id =  {i: category_id for category_id, i in unique_category_id_to_id.items()}
            
            for i, segment in enumerate(result['segments_info']):
                result['segments_info'][i]["new_id"] = unique_category_id_to_id[result['segments_info'][i]["category_id"]]
            
            # Update original panoptic_seg_id array with new ids as the new segmentation combines different categories.
            custom_panoptic_seg_id = np.zeros((panoptic_seg_id.shape[0], panoptic_seg_id.shape[1]), dtype=np.uint8)
            
            # Update this custom panoptic seg matrix
            for i, segment in enumerate(result['segments_info']):
                custom_panoptic_seg_id[result['segments_info'][i]['id'] == panoptic_seg_id] = result['segments_info'][i]['new_id']
                
            # Create new Segmentation info
            # [{'area': 243, 'category_id': 3, 'id': 0, 'isthing': True},
            #   {'area': 730578, 'category_id': 184, 'id': 1, 'isthing': False}]
            
            custom_panoptic_segments_info = []
            for category_id in unique_category_id:
                custom_panoptic_segments_info.append({
                    'segment_id': unique_category_id_to_id[category_id], 
                    'category_id': category_id,
                    'bbox': [],
                    'area': 0,
                    'iscroud': 0,
                    'isthing': 0
                })

            # annotations of our construction things
            omask = processing_data['annotations']
            
            TEMP_ANNOTATIONS = []
            
            # Overlay things mask one at a time
            for annotation in omask:
                # overlay mask of construction things on top of detr output
                omask_image_id = overlay_custom_mask.get_overlayed_mask((h, w), annotation)
                custom_panoptic_seg_id[omask_image_id.astype(np.bool_)] = custom_panoptic_seg_id.max() + 1
                custom_panoptic_segments_info.append({
                    'segment_id': custom_panoptic_seg_id.max(), 
                    'category_id': cat2id[category_name], 
                    'bbox': annotation['bbox'],
                    'area': annotation['area'],
                    'iscroud': 0,
                    'isthing': 1
                })

                # append annotation of construction things in json file
                annotation["category_id"] = cat2id[category_name]
                annotation["image_id"] = image_id
                TEMP_ANNOTATIONS.append(annotation)
            
            # Convert to binary segment
            binary_masks = np.zeros((
                custom_panoptic_seg_id.max() + 1,
                custom_panoptic_seg_id.shape[0],
                custom_panoptic_seg_id.shape[1]),
                dtype=np.uint8
            )
                
            # for each binary mask, detect contours and create annotation for those contours
            if len(unique_category_id):
                # Skip the onse which are added by us
                for category_id in unique_category_id:
                    binary_masks[unique_category_id_to_id[category_id], :, :] = custom_panoptic_seg_id == unique_category_id_to_id[category_id]
                    annotation_info = convert_to_coco.main(binary_masks[unique_category_id_to_id[category_id]], None, image_id, category_id, unique_category_id_to_id[category_id], 0)
                    if annotation_info is not None:
                        annotation_info["image_id"] = image_id
                        annotation_info["category_id"] = category_id
                        TEMP_ANNOTATIONS.append(annotation_info)
                        
                        custom_panoptic_segments_info[unique_category_id_to_id[category_id]]['bbox'] = annotation_info['bbox']
                        custom_panoptic_segments_info[unique_category_id_to_id[category_id]]['area'] = annotation_info['area']
            else:
                # Do something for the once where there are no predictions
                ## Probably mark them as None
                pass
            
            

            # Write data to global json and save files to image dir's
            
            # save image to new path as .jpg
            imo.save(output_file_path)
            
            # save panoptic image
            Image.fromarray(id2rgb(custom_panoptic_seg_id), 'RGB').save(output_mask_path)

            # create image_info object and append it to original list
            image_info = coco_creator_tools.create_image_info(image_id, output_file_name, imo.size)
            image_info["original_file"] = processing_file

            GLOBAL_COCO["images"].append(image_info)
            GLOBAL_PANOPTIC["images"].append(image_info)

            for annotation in TEMP_ANNOTATIONS:
                annotation["id"] = annotation_id
                GLOBAL_COCO["annotations"].append(annotation)
                annotation_id += 1
                
            for segment_info in custom_panoptic_segments_info:
                segment_info["id"] = segment_id
                segment_id += 1
                
            GLOBAL_PANOPTIC["annotations"].append({
                "segments_info": custom_panoptic_segments_info,
                "file_name": output_mask_name,
                "image_id": image_id
            })

            # increment the image_count
            image_id += 1
        
        except Exception as e:
            # if there is any error, add info about it in errros file and procees to next image
            print("Error occurred while processig file:", processing_file)
            
            with open(os.path.join(ROOT_DIR, "error.json"), 'r') as error_file:
                error_json = json.load(error_file)
                
            with open(os.path.join(ROOT_DIR, "error.json"), 'w') as error_file:
                error_json["error"].append({
                    "processing_file": processing_file,
                    "processing_data": processing_data
                })
                
                json.dump(error_json, error_file)
                
            traceback.print_exc()
    
    total_time_str = str(datetime.timedelta(seconds=int(time.time() - start)))
    print(f"Completed Category: {category_name}, Time Taken: {total_time_str}")

    # open the final json, and commit changes in that file
    with open(os.path.join(ROOT_DIR, "coco.json"), 'w') as output_json_file:
        json.dump(GLOBAL_COCO, output_json_file)
        
    with open(os.path.join(ROOT_DIR, "panoptic.json"), 'w') as output_json_file:
        json.dump(GLOBAL_PANOPTIC, output_json_file, default=convert)
        
    print(image_id, annotation_id, segment_id)

Processing Category: hydra_crane
Completed Category: hydra_crane, Time Taken: 0:00:47
101 416 418
Processing Category: split_units
Completed Category: split_units, Time Taken: 0:05:40
639 2083 2097
Processing Category: wheel_loader
Completed Category: wheel_loader, Time Taken: 0:01:27
789 2777 2799
Processing Category: dump_truck_tipper_truck
Completed Category: dump_truck_tipper_truck, Time Taken: 0:00:47
889 3263 3285
Processing Category: transit_mixer
Completed Category: transit_mixer, Time Taken: 0:00:27
939 3512 3534
Processing Category: glass_wool
Completed Category: glass_wool, Time Taken: 0:00:37
1009 3820 3842
Processing Category: rmu_units
Completed Category: rmu_units, Time Taken: 0:00:33
1109 4065 4088
Processing Category: chiller
Completed Category: chiller, Time Taken: 0:00:22
1154 4215 4242
Processing Category: adhesives
Completed Category: adhesives, Time Taken: 0:00:34
1254 4423 4450
Processing Category: structural_steel_channel


/home/ammar/projects/.venv/lib/python3.6/site-packages/PIL/Image.py:963: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Completed Category: structural_steel_channel, Time Taken: 0:02:38
1655 5320 5348
Processing Category: sanitary_fixtures
Completed Category: sanitary_fixtures, Time Taken: 0:07:28
2176 7990 8020
Processing Category: aac_blocks
Completed Category: aac_blocks, Time Taken: 0:03:20
2439 9026 9057
Processing Category: control_panel
Completed Category: control_panel, Time Taken: 0:02:10
2684 9714 9758
Processing Category: vrf_units
Completed Category: vrf_units, Time Taken: 0:00:33
2734 9836 9880
Processing Category: wood_primer
Completed Category: wood_primer, Time Taken: 0:00:05
2746 9865 9910
Processing Category: rcc_hume_pipes
Completed Category: rcc_hume_pipes, Time Taken: 0:02:21
2986 10577 10625
Processing Category: hoist
Completed Category: hoist, Time Taken: 0:06:14
3521 12571 12641
Processing Category: emulsion_paint
Completed Category: emulsion_paint, Time Taken: 0:00:16
3553 12650 12722
Processing Category: threaded_rod
Error occurred while processig file: /home/ammar/data/constru

Traceback (most recent call last):
  File "<ipython-input-30-d082edade9da>", line 72, in <module>
    imo = Image.open(processing_file).convert('RGB')
  File "/home/ammar/projects/.venv/lib/python3.6/site-packages/PIL/Image.py", line 2912, in open
    fp = builtins.open(filename, "rb")
FileNotFoundError: [Errno 2] No such file or directory: '/home/ammar/data/construction/threaded_rod/images/img_052.jpg'


Error occurred while processig file: /home/ammar/data/construction/threaded_rod/images/img_050.jpg


Traceback (most recent call last):
  File "<ipython-input-30-d082edade9da>", line 72, in <module>
    imo = Image.open(processing_file).convert('RGB')
  File "/home/ammar/projects/.venv/lib/python3.6/site-packages/PIL/Image.py", line 2912, in open
    fp = builtins.open(filename, "rb")
FileNotFoundError: [Errno 2] No such file or directory: '/home/ammar/data/construction/threaded_rod/images/img_050.jpg'


Completed Category: threaded_rod, Time Taken: 0:18:18
4104 13989 14061
Processing Category: switch_boards_and_switches
Completed Category: switch_boards_and_switches, Time Taken: 0:03:22
4604 15025 15130
Processing Category: smoke_detectors
Completed Category: smoke_detectors, Time Taken: 0:00:30
4654 15119 15224
Processing Category: vcb_panel
Completed Category: vcb_panel, Time Taken: 0:04:55
5154 16808 16934
Processing Category: rmc_batching_plant
Completed Category: rmc_batching_plant, Time Taken: 0:00:30
5213 17108 17239
Processing Category: fine_aggregate
Completed Category: fine_aggregate, Time Taken: 0:17:13
5713 18088 18221
Processing Category: hollow_concrete_blocks
Completed Category: hollow_concrete_blocks, Time Taken: 0:00:19
5763 18248 18382
Processing Category: junction_box
Completed Category: junction_box, Time Taken: 0:00:21
5815 18388 18523
Processing Category: marble
Completed Category: marble, Time Taken: 0:00:24
5865 18573 18711
Processing Category: pipe_fittings
Co

Traceback (most recent call last):
  File "<ipython-input-30-d082edade9da>", line 72, in <module>
    imo = Image.open(processing_file).convert('RGB')
  File "/home/ammar/projects/.venv/lib/python3.6/site-packages/PIL/Image.py", line 2912, in open
    fp = builtins.open(filename, "rb")
FileNotFoundError: [Errno 2] No such file or directory: '/home/ammar/data/construction/skid_steer_loader/images/img_105.jpg'


Completed Category: skid_steer_loader, Time Taken: 0:01:44
9166 31877 32092
Processing Category: enamel_paint
Completed Category: enamel_paint, Time Taken: 0:00:19
9217 31997 32214
Processing Category: texture_paint
Completed Category: texture_paint, Time Taken: 0:00:19
9269 32100 32319
Processing Category: fire_extinguishers
Completed Category: fire_extinguishers, Time Taken: 0:01:36
9497 32726 32948
Processing Category: aluminium_frames_for_false_ceiling
Completed Category: aluminium_frames_for_false_ceiling, Time Taken: 0:00:26
9547 32868 33091
Processing Category: hot_mix_plant
Completed Category: hot_mix_plant, Time Taken: 0:00:57
9647 33342 33570
Processing Category: cu_piping
Completed Category: cu_piping, Time Taken: 0:03:03
10147 34720 34955
